## Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset. 
Multi Channel CNN Model, Training and Testing Notebook.
Code by M. Siino. 

From the paper: "Automated Detection of Fake News Spreaders: An Evaluative Study of Transformers and SOTA Models on Multilingual Dataset." by M.Siino et al.

## Importing modules.

In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from io import open
from pathlib import Path

## Importing DS and extract in current working directory.

In [2]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
urlTrainingSet = "https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-training-2020-02-23.zip"
urlTestSet="https://github.com/marco-siino/fake_news_spreaders_detection/raw/main/dataset/pan20-author-profiling-test-2020-02-23.zip"

training_set = tf.keras.utils.get_file("pan20-author-profiling-training-2020-02-23.zip", urlTrainingSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')
test_set = tf.keras.utils.get_file("pan20-author-profiling-test-2020-02-23.zip", urlTestSet,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

training_set_dir = os.path.join(os.path.dirname(training_set), 'pan20-author-profiling-training-2020-02-23')
test_set_dir = os.path.join(os.path.dirname(test_set), 'pan20-author-profiling-test-2020-02-23')

print(training_set)
print(training_set_dir)

!ls -A

  16384/3094459 [..............................] - ETA: 0s

 638976/3094459 [=====>........................] - ETA: 0s

3096576/3094459 [==============================] - 0s 0us/step


3104768/3094459 [==============================] - 0s 0us/step


  16384/2135236 [..............................] - ETA: 0s

 704512/2135236 [========>.....................] - ETA: 0s

2138112/2135236 [==============================] - 0s 0us/step


2146304/2135236 [==============================] - 0s 0us/step


./pan20-author-profiling-training-2020-02-23.zip
./pan20-author-profiling-training-2020-02-23


__MACOSX
__notebook__.ipynb
pan20-author-profiling-test-2020-02-23
pan20-author-profiling-test-2020-02-23.zip
pan20-author-profiling-training-2020-02-23
pan20-author-profiling-training-2020-02-23.zip


## Build folders hierarchy to use Keras folders preprocessing function.

In [3]:
### Training Folders. ###

# First level directory.
if not os.path.exists('train_dir_en'):
    os.makedirs('train_dir_en')
if not os.path.exists('train_dir_es'):
    os.makedirs('train_dir_es')

# Class labels directory.
if not os.path.exists('train_dir_en/0'):
    os.makedirs('train_dir_en/0')
if not os.path.exists('train_dir_es/0'):
    os.makedirs('train_dir_es/0')
if not os.path.exists('train_dir_en/1'):
    os.makedirs('train_dir_en/1')
if not os.path.exists('train_dir_es/1'):
    os.makedirs('train_dir_es/1')

# Make Py variables.
train_dir='train_dir_'

## Test Folders. ##
# First level directory.
if not os.path.exists('test_dir_en'):
    os.makedirs('test_dir_en')
if not os.path.exists('test_dir_es'):
    os.makedirs('test_dir_es')

# Class labels directory.
if not os.path.exists('test_dir_en/0'):
    os.makedirs('test_dir_en/0')
if not os.path.exists('test_dir_es/0'):
    os.makedirs('test_dir_es/0')
if not os.path.exists('test_dir_en/1'):
    os.makedirs('test_dir_en/1')
if not os.path.exists('test_dir_es/1'):
    os.makedirs('test_dir_es/1')

# Make Py variables.
test_dir='test_dir_'

!ls -A

__MACOSX
__notebook__.ipynb
pan20-author-profiling-test-2020-02-23
pan20-author-profiling-test-2020-02-23.zip
pan20-author-profiling-training-2020-02-23
pan20-author-profiling-training-2020-02-23.zip
test_dir_en
test_dir_es
train_dir_en
train_dir_es


## Set language and directory paths.


In [4]:
# Set en and es train_dir and test_dir paths.
language='es'

truth_file_training_dir_es=training_set_dir+'/'+language+'/'
truth_file_training_path_es = truth_file_training_dir_es+'truth.txt'

truth_file_test_dir=test_set_dir
truth_file_test_path_es = truth_file_test_dir+'/'+language+'.txt'


language='en'

truth_file_training_dir_en=training_set_dir+'/'+language+'/'
truth_file_training_path_en = truth_file_training_dir_en+'truth.txt'

truth_file_test_path_en = truth_file_test_dir+'/'+language+'.txt'

## Read truth.txt to organize training dataset folders.



In [5]:
# Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_en+fNameXml):
      os.rename(truth_file_training_dir_en+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

language='es'
# Organize ES folders.
# Open the file truth.txt with read only permit.
f = open(truth_file_training_path_es, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_training_dir_es+fNameXml):
      os.rename(truth_file_training_dir_es+fNameXml, './train_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Read truth.txt to organize test dataset folders.

In [6]:
#Organize EN folders.
language='en'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_en, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

#Organize EN folders.
language='es'
# Open the file truth.txt with read only permit.
f = open(truth_file_test_path_es, "r")
# use readline() to read the first line 
line = f.readline()
# use the read line to read further.
# If the file is not empty keep reading one line
# at a time, till the file is empty
while line:
    # Split line at :::
    x = line.split(":::")
    fNameXml = x[0]+'.xml'
    fNameTxt = x[0]+'.txt'
    # Second coord [0] gets just the first character (label) and not /n too.
    label = x[1][0]

    # Now move the file to the right folder.
    if os.path.exists(truth_file_test_dir+'/'+language+'/'+fNameXml):
      os.rename(truth_file_test_dir+'/'+language+'/'+fNameXml, './test_dir_'+language+'/'+label+'/'+fNameTxt )

    # use readline() to read next line
    line = f.readline()

## Function to pre-process source text.

In [7]:
def custom_standardization(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_es_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="es">', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_author_lang_es_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

## Building the dataset.

In [8]:
batch_size=1

# Build the dataset for Spanish.
language='es'

raw_train_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size, 
    #validation_split=0.0, 
    #subset='training', 
    shuffle='false',
    seed=1
    )

raw_test_ds_es = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle='false'
    )


# Build the dataset for Spanish.
language='en'

raw_train_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir+language, 
    batch_size=batch_size, 
    #validation_split=0.0, 
    #subset='training', 
    shuffle='false',
    seed=1
    )

raw_test_ds_en = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir+language, 
    batch_size=batch_size,
    shuffle='false'
    )


Found 300 files belonging to 2 classes.


2022-03-24 22:23:16.466273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 22:23:16.561062: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 22:23:16.561795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 22:23:16.564950: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 200 files belonging to 2 classes.
Found 300 files belonging to 2 classes.


Found 200 files belonging to 2 classes.


## First model's layer: Text Vectorization.

In [9]:
# Maximum number of words allowed 80000 in our dictionary.
max_features = 80000
# After tokenization 4500 covers all the document lenghts in our dataset.
sequence_length = 4500

vectorize_layer_es = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

vectorize_layer_en = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_text = raw_train_ds_en.map(lambda x, y: x)
vectorize_layer_en.adapt(train_text)

train_text = raw_train_ds_es.map(lambda x, y: x)
vectorize_layer_es.adapt(train_text)

2022-03-24 22:23:19.615365: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Some training parameters...

In [10]:
# Word embedding dimensions.
embedding_dim = 500

num_runs = 5 
num_epochs_per_run = 100

opt = tf.keras.optimizers.RMSprop()

## Functions to create and compile models

In [11]:
def create_compile_model_en():
    input= tf.keras.Input(shape=(1,), dtype=tf.string)
    x=vectorize_layer_en(input)
  
    y1=layers.Embedding(len(vectorize_layer_en.get_vocabulary()) + 1, embedding_dim)(x)                     
    y1=layers.Dropout(0.8)(y1)

    y2=layers.Embedding(len(vectorize_layer_en.get_vocabulary()) + 1, embedding_dim)(x)                     
    y2=layers.Dropout(0.8)(y2)

    y3=layers.Embedding(len(vectorize_layer_en.get_vocabulary()) + 1, embedding_dim)(x)                     
    y3=layers.Dropout(0.8)(y3)

    y4=layers.Embedding(len(vectorize_layer_en.get_vocabulary()) + 1, embedding_dim)(x)                     
    y4=layers.Dropout(0.8)(y4)

    y1=layers.Conv1D(32,32)(y1)
    y1=layers.MaxPooling1D()(y1)
    y1=layers.Dropout(0.5)(y1)
    y1=layers.Flatten()(y1)

    y2=layers.Conv1D(32,16)(y2)
    y2=layers.MaxPooling1D()(y2)
    y2=layers.Dropout(0.5)(y2)
    y2=layers.Flatten()(y2)

    y3=layers.Conv1D(32,2)(y3)
    y3=layers.MaxPooling1D()(y3)
    y3=layers.Dropout(0.5)(y3)
    y3=layers.Flatten()(y3)

    y4=layers.Conv1D(32,1)(y4)
    y4=layers.MaxPooling1D()(y4)
    y4=layers.Dropout(0.5)(y4)
    y4=layers.Flatten()(y4)

    merged = layers.concatenate([y1,y2,y3,y4])                         
  
    x=layers.Dense(300)(merged)
    x=layers.Dropout(0.5)(x)
    output=layers.Dense(1)(x)
    model_en = tf.keras.Model(inputs=input, outputs=output)
    model_en.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer=opt, metrics=tf.metrics.BinaryAccuracy(threshold=0.0)) 
    
    return model_en

def create_compile_model_es():
    input= tf.keras.Input(shape=(1,), dtype=tf.string)
    x=vectorize_layer_es(input)
  
    y1=layers.Embedding(len(vectorize_layer_es.get_vocabulary()) + 1, embedding_dim)(x)                     
    y1=layers.Dropout(0.8)(y1)

    y2=layers.Embedding(len(vectorize_layer_es.get_vocabulary()) + 1, embedding_dim)(x)                     
    y2=layers.Dropout(0.8)(y2)

    y3=layers.Embedding(len(vectorize_layer_es.get_vocabulary()) + 1, embedding_dim)(x)                     
    y3=layers.Dropout(0.8)(y3)

    y4=layers.Embedding(len(vectorize_layer_es.get_vocabulary()) + 1, embedding_dim)(x)                     
    y4=layers.Dropout(0.8)(y4)

    y1=layers.Conv1D(32,32)(y1)
    y1=layers.MaxPooling1D()(y1)
    y1=layers.Dropout(0.5)(y1)
    y1=layers.Flatten()(y1)

    y2=layers.Conv1D(32,16)(y2)
    y2=layers.MaxPooling1D()(y2)
    y2=layers.Dropout(0.5)(y2)
    y2=layers.Flatten()(y2)

    y3=layers.Conv1D(32,2)(y3)
    y3=layers.MaxPooling1D()(y3)
    y3=layers.Dropout(0.5)(y3)
    y3=layers.Flatten()(y3)

    y4=layers.Conv1D(32,1)(y4)
    y4=layers.MaxPooling1D()(y4)
    y4=layers.Dropout(0.5)(y4)
    y4=layers.Flatten()(y4)

    merged = layers.concatenate([y1,y2,y3,y4])                         
  
    x=layers.Dense(300)(merged)
    x=layers.Dropout(0.5)(x)
    output=layers.Dense(1)(x)
    model_es = tf.keras.Model(inputs=input, outputs=output)
    model_es.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer=opt, metrics=tf.metrics.BinaryAccuracy(threshold=0.0))    
    
    return model_es

## Training and evaluation of English model

In [12]:
runs_accuracy = []
for run in range(1,(num_runs+1)):
    epochs_accuracy=[]
    model_en = create_compile_model_en()
    
    for epoch in range (0,num_epochs_per_run):
        history = model_en.fit(
          raw_train_ds_en,
          validation_data = raw_test_ds_en,
          epochs=1,
          shuffle=False,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )
        accuracy = history.history['val_binary_accuracy']
        print("Run: ",run,"/ Accuracy at epoch ",epoch," is: ", accuracy[0],"\n")
        epochs_accuracy.append(accuracy[0])

    print("Accuracies over epochs:",epochs_accuracy,"\n\n")
    runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies on English are:", runs_accuracy)
print("The median for English is:",runs_accuracy[2],"\n\n\n")


2022-03-24 22:23:24.357327: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


  1/300 [..............................] - ETA: 38:26 - loss: 0.6140 - binary_accuracy: 1.0000

  3/300 [..............................] - ETA: 12s - loss: 245.5783 - binary_accuracy: 0.3333

  5/300 [..............................] - ETA: 12s - loss: 380.3228 - binary_accuracy: 0.2000

  7/300 [..............................] - ETA: 12s - loss: 306.9463 - binary_accuracy: 0.2857

...

 85/300 [=======>......................] - ETA: 8s - loss: 1.7115 - binary_accuracy: 0.9176

 87/300 [=======>......................] - ETA: 8s - loss: 1.6721 - binary_accuracy: 0.9195

 89/300 [=======>......................] - ETA: 8s - loss: 1.6345 - binary_accuracy: 0.9213

 91/300 [========>.....................] - ETA: 8s - loss: 1.5986 - binary_accuracy: 0.9231

 93/300 [========>.....................] - ETA: 8s - loss: 1.5642 - binary_accuracy: 0.9247

## Training and evaluation of Spanish model

In [ ]:
runs_accuracy = []
for run in range(1,(num_runs+1)):
    epochs_accuracy=[]
    model_es = create_compile_model_es()
    
    for epoch in range (0,num_epochs_per_run):
        history = model_es.fit(
          raw_train_ds_es,
          validation_data = raw_test_ds_es,
          epochs=1,
          shuffle=False,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )
        accuracy = history.history['val_binary_accuracy']
        print("Run: ",run,"/ Accuracy at epoch ",epoch," is: ", accuracy[0],"\n")
        epochs_accuracy.append(accuracy[0])

    print("Accuracies over epochs:",epochs_accuracy,"\n\n")
    runs_accuracy.append(max(epochs_accuracy))

runs_accuracy.sort()
print("\n\n Over all runs maximum accuracies on Spanish are:", runs_accuracy)
print("The median for SpanishEnglish is:",runs_accuracy[2],"\n\n\n")
